In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import json
import xml.etree.ElementTree as et
import nltk
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.corpus import stopwords
from nltk.collocations import *
import os
import sys

C:\Anaconda2\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [3]:
reload(sys)  
sys.setdefaultencoding('utf8')

In [4]:
#load xml to dataframe
tree = et.parse("../data/AviationData.xml")
root = tree.getroot()
events_list = [i.attrib for i in root[0]]
df_events = pd.DataFrame(events_list)
df_events.head()

,AccidentNumber,AirCarrier,AircraftCategory,AircraftDamage,AirportCode,AirportName,AmateurBuilt,BroadPhaseOfFlight,Country,EngineType,...,PublicationDate,PurposeOfFlight,RegistrationNumber,ReportStatus,Schedule,TotalFatalInjuries,TotalMinorInjuries,TotalSeriousInjuries,TotalUninjured,WeatherCondition
0,GAA15CA244,,,,,,,,United States,,...,,,N786AB,Preliminary,,,,,,
1,CEN15LA392,,Airplane,Substantial,IWS,WEST HOUSTON,No,LANDING,United States,,...,09/04/2015,Instructional,N452CS,Preliminary,,,,,2,VMC
2,GAA15CA246,,,,,,,,United States,,...,,,N112RP,Preliminary,,,,,,
3,GAA15CA243,,,,,,,,United States,,...,,,N8404X,Preliminary,,,,,,
4,GAA15CA242,,,,,,,,United States,,...,,,N5255G,Preliminary,,,,,,


In [5]:
nar_list = []
filelist = [name for name in os.listdir("../data/") if name.endswith(".json")]
for f in filelist:
    with open(os.path.join("../data/", f)) as json_file:
        j = json.load(json_file)['data']
        nar_list += j
df_nar = pd.DataFrame(nar_list)

In [6]:
df_combined = pd.merge(df_events, df_nar,how='left',on='EventId')

In [7]:
df_combined.shape

(77257, 33)

In [8]:
#filter only records with probable cause
df_pc = df_combined[(df_combined['probable_cause']!='')]

In [9]:
df_pc.shape

(50503, 33)

In [10]:
df_pc.head()

,AccidentNumber,AirCarrier,AircraftCategory,AircraftDamage,AirportCode,AirportName,AmateurBuilt,BroadPhaseOfFlight,Country,EngineType,...,RegistrationNumber,ReportStatus,Schedule,TotalFatalInjuries,TotalMinorInjuries,TotalSeriousInjuries,TotalUninjured,WeatherCondition,narrative,probable_cause
175,GAA15CA208,,Airplane,Substantial,OVO,NORTH VERNON,No,LANDING,United States,Reciprocating,...,N133VU,Probable Cause,,,,,1,VMC,NTSB investigators used data provided by vario...,The pilot's failure to maintain pitch control ...
183,GAA15CA197,,Helicopter,Substantial,,N/A,No,LANDING,United States,Reciprocating,...,N58397,Probable Cause,,,,,2,VMC,NTSB investigators used data provided by vario...,The non-helicopter rated pilot's improper land...
203,GAA15CA189,,Airplane,Substantial,U660,Hidden Splendor,No,TAKEOFF,United States,Reciprocating,...,N735JB,Probable Cause,,,,,2,VMC,NTSB investigators used data provided by vario...,The pilot's failure to maintain directional co...
205,ERA15CA272,,Airplane,Substantial,DVK,Stuart Powell Field,Yes,GO-AROUND,United States,Reciprocating,...,N31CS,Probable Cause,,,1,,,VMC,NTSB investigators used data provided by vario...,"The pilot's inaccurate fuel planning, which re..."
217,WPR15CA214,,Airplane,Substantial,K69S,Avey Field,No,LANDING,United States,Reciprocating,...,C-GDYN,Probable Cause,,,1,,,VMC,NTSB investigators used data provided by vario...,The pilot's inadequate decision to continue th...


## Overall Text Analysis

In [11]:
pc_list = df_pc['probable_cause'].tolist()
pc_list = [i for i in pc_list]
pc_string = str(''.join(pc_list))
pc_string = unicode(pc_string, errors='replace')

In [12]:
pc_string = pc_string.replace(u"\u2018", "'").replace(u"\u2019", "'")

In [13]:
sw = stopwords.words('english')
tokens = nltk.word_tokenize(pc_string)
text = nltk.Text(tokens)
words = [w.lower() for w in text if w.isalpha()]
words = [w for w in words if w not in sw]

In [14]:
#words and bigram
fd = nltk.FreqDist(words)
bi_fd = nltk.FreqDist(nltk.bigrams(words))

In [15]:
fd.most_common(30)

[(u'pilot', 41897),
 (u'failure', 28745),
 (u'landing', 17154),
 (u'maintain', 15560),
 (u'control', 12881),
 (u'loss', 12856),
 (u'resulted', 12614),
 (u'accident', 12037),
 (u'engine', 11589),
 (u'factor', 10112),
 (u'power', 9648),
 (u'fuel', 9482),
 (u'inadequate', 9131),
 (u'factors', 9069),
 (u'contributing', 8705),
 (u'flight', 8367),
 (u'airplane', 8048),
 (u'improper', 7994),
 (u'due', 7596),
 (u'terrain', 7433),
 (u'conditions', 6257),
 (u'aircraft', 5752),
 (u'directional', 5461),
 (u'resulting', 4973),
 (u'lack', 4789),
 (u'takeoff', 4635),
 (u'runway', 4552),
 (u'undetermined', 4339),
 (u'airspeed', 4004),
 (u'subsequent', 3827)]

In [16]:
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [17]:
finder = BigramCollocationFinder(fd, bi_fd)

In [18]:
finder.nbest(bigram_measures.raw_freq,30)

[(u'pilot', u'failure'),
 (u'failure', u'maintain'),
 (u'loss', u'engine'),
 (u'engine', u'power'),
 (u'directional', u'control'),
 (u'contributing', u'accident'),
 (u'maintain', u'directional'),
 (u'pilot', u'inadequate'),
 (u'pilot', u'improper'),
 (u'accident', u'pilot'),
 (u'power', u'due'),
 (u'forced', u'landing'),
 (u'failure', u'pilot'),
 (u'maintain', u'adequate'),
 (u'landing', u'gear'),
 (u'student', u'pilot'),
 (u'undetermined', u'reasons'),
 (u'control', u'airplane'),
 (u'factor', u'accident'),
 (u'fuel', u'exhaustion'),
 (u'control', u'landing'),
 (u'power', u'undetermined'),
 (u'contributing', u'factor'),
 (u'pilot', u'maintain'),
 (u'inadequate', u'preflight'),
 (u'contributing', u'factors'),
 (u'aircraft', u'control'),
 (u'loss', u'control'),
 (u'resulted', u'loss'),
 (u'maintain', u'clearance')]

While some of the bigram above doesn't exactly make sense grammatically, this happens because of the removal of stopwords. (aircraft,control) originally was most likely to be "aircraft out of control".

## Apply tokenization to Dataframe

In [19]:
#create a common tokenization function
def colTokenize(x):
    text = nltk.Text(nltk.word_tokenize(x.replace(u"\u2018", "'").replace(u"\u2019", "'")))
    #remove numeric and punctuation
    words = [w.lower() for w in text if w.isalpha()]
    #remove stopwords
    words = [w for w in words if w not in sw]
    return words

In [20]:
df_pc['pc_tokens'] = df_pc['probable_cause'].apply(lambda x: colTokenize(x))

In [21]:
df_pc['pc_bigram'] = df_pc['pc_tokens'].apply(lambda x: list(nltk.bigrams(x)))

In [22]:
df_pc['pc_bigram'].head()

175    [(pilot, failure), (failure, maintain), (maint...
183    [(rated, pilot), (pilot, improper), (improper,...
203    [(pilot, failure), (failure, maintain), (maint...
205    [(pilot, inaccurate), (inaccurate, fuel), (fue...
217    [(pilot, inadequate), (inadequate, decision), ...
Name: pc_bigram, dtype: object

Based on the bigram generated from the overall text, accidents where pilot error is a contributing factor can be detected based on the phrase "pilot failure","pilot inadequate", and "pilot improper".

In [23]:
def isinwords(x,words):
    found = [b for b in x if b in words]
    if found:
        return 1

In [24]:
#pc_pilot = [('pilot','inadequate'),('pilot','failure'),('pilot','improper')]
#test for token
df_pc['factor_human'] = df_pc['pc_tokens'].apply(lambda x: isinwords(x,['pilot','pilots','instructor']))

In [25]:
df_pc[(df_pc['factor_human']==1)]['factor_human'].sum()

36048.0

### Finding
Out of 50,503 records of probable cause, human factor is involved in 36,048 (71%) of them!  